# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [48]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import gmaps

# Import API key
from api_keys import gmaps_key
from api_keys import geoapify_key




In [49]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-tsilma,65.4410,52.1498,17.22,95,96,5.79,RU,1678555180
1,1,jamestown,42.0970,-79.2353,31.64,87,100,8.05,US,1678555181
2,2,hanstholm,57.1167,8.6167,35.17,56,59,27.27,DK,1678555181
3,3,severo-kurilsk,50.6789,156.1250,29.82,88,99,6.67,RU,1678555182
4,4,atuona,-9.8000,-139.0333,79.48,76,18,8.16,PF,1678555184


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:
##%%capture --no-display

# Configure the map plot
gmaps.configure(api_key= gmaps_key)
base = gmaps.figure(center= (0, 0), zoom_level= 2)
heat_layer = gmaps.heatmap_layer(city_data_df[["Lat", "Lng"]], weights= city_data_df["Humidity"], max_intensity= 75, point_radius= 5)
base.add_layer(heat_layer)

# Display the map
base

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= 90) &
                                  (city_data_df["Max Temp"] >=70) &
                                  (city_data_df["Humidity"] <= 80) &
                                  (city_data_df["Humidity"] >= 60) &
                                  (city_data_df["Cloudiness"] <= 25)
                                  ]

# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head(25)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,atuona,-9.8000,-139.0333,79.48,76,18,8.16,PF,1678555184
46,46,carnarvon,-24.8667,113.6333,75.52,76,0,23.89,AU,1678555231
51,51,savannah bight,16.4500,-85.8500,78.19,70,1,11.30,HN,1678555238
123,123,beloha,-25.1667,45.0500,74.37,75,4,15.61,MG,1678555320
157,157,porto novo,6.4965,2.6036,84.02,70,25,7.52,BJ,1678555363
162,162,doha,25.2867,51.5333,73.38,68,0,14.97,QA,1678555368
164,164,port hedland,-20.3167,118.5667,84.25,79,0,10.36,AU,1678555370
191,191,puerto escondido,15.8500,-97.0667,88.63,66,0,6.91,MX,1678555400
233,233,guaruja,-23.9931,-46.2564,82.26,78,20,5.01,BR,1678555445
245,245,kochi,9.9399,76.2602,82.38,70,17,2.15,IN,1678555460


### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,atuona,PF,-9.8000,-139.0333,76,
46,carnarvon,AU,-24.8667,113.6333,76,
51,savannah bight,HN,16.4500,-85.8500,70,
123,beloha,MG,-25.1667,45.0500,75,
157,porto novo,BJ,6.4965,2.6036,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel
radius = 10000
coordinates = ""

params = {
    "categories": f"accomodation.hotel",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    coordinates = f"{row['Lat']}, {row['Lng']}"

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = f"{base_url}?categories={params['categories']}&filter={params['filter']}&bias={params['bias']}&apiKey={params['key']}"

    #Convert the API response to JSON format
    name_address = requests.get(name_address).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atuona - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
savannah bight - nearest hotel: No hotel found
beloha - nearest hotel: No hotel found
porto novo - nearest hotel: No hotel found
doha - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
puerto escondido - nearest hotel: No hotel found
guaruja - nearest hotel: No hotel found
kochi - nearest hotel: No hotel found
dhidhdhoo - nearest hotel: No hotel found
olinda - nearest hotel: No hotel found
grand gaube - nearest hotel: No hotel found
alappuzha - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
puri - nearest hotel: No hotel found
cam ranh - nearest hotel: No hotel found
arica - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
cuyo - nearest hotel: No hotel found
sisophon - nearest hotel: No hotel found
poum - nearest hotel: No hotel found
muli - nearest hotel: No hotel found
eyl - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,atuona,PF,-9.8000,-139.0333,76,No hotel found
46,carnarvon,AU,-24.8667,113.6333,76,No hotel found
51,savannah bight,HN,16.4500,-85.8500,70,No hotel found
123,beloha,MG,-25.1667,45.0500,75,No hotel found
157,porto novo,BJ,6.4965,2.6036,70,No hotel found
162,doha,QA,25.2867,51.5333,68,No hotel found
164,port hedland,AU,-20.3167,118.5667,79,No hotel found
191,puerto escondido,MX,15.8500,-97.0667,66,No hotel found
233,guaruja,BR,-23.9931,-46.2564,78,No hotel found
245,kochi,IN,9.9399,76.2602,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
##%%capture --no-display

# Configure the map plot
base = gmaps.figure(center= (0, 0), zoom_level= 2)
heat_layer = gmaps.heatmap_layer(hotel_df[["Lat", "Lng"]], weights= hotel_df["Humidity"], max_intensity= 75, point_radius= 5)
info_layer = gmaps.marker_layer(hotel_df[["Lat", "Lng"]], hover_text= hotel_df[["Hotel Name", "Country"]])
base.add_layer(heat_layer)
base.add_layer(info_layer)


# Display the map
base